<a href="https://colab.research.google.com/github/zombienok/short_quest_generator/blob/main/quest_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!git clone https://github.com/zombienok/short_quest_generator.git
!cd short_quest_generator
!pip install -r /content/short_quest_generator/requirements.txt
!curl -fsSL https://ollama.com/install.sh | sh

fatal: destination path 'short_quest_generator' already exists and is not an empty directory.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [48]:
import subprocess
import time
import ollama


# Запускаем сервер Ollama в фоне (если ещё не запущен)
proc = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ждём, пока сервер запустится (обычно 3-5 секунд)
time.sleep(3)

# Проверяем, работает ли сервер
try:
    ollama.pull("owl/t-lite:latest")
    print(ollama.list())  # Должен вывести список моделей (пустой, если ничего не загружено)
except Exception as e:
    print("Ошибка:", e)

models=[Model(model='owl/t-lite:latest', modified_at=datetime.datetime(2025, 8, 4, 1, 58, 8, 982281, tzinfo=TzInfo(UTC)), digest='52bcfdc27e1b605bb7905cfbfaca2cf5541e85f86c1e1138f0e00d11336242b1', size=4661229593, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


In [39]:
import ollama
import json


MODEL_TO_USE = "owl/t-lite:latest"


messages_strict = [{
    "role": "system",
    "content": "Ты генератор структуры для квеста. Отвечаешь ТОЛЬКО в указанном формате. Никаких комментариев, пояснений, символов *, -, >."
}]
# вводные данные
COUNT_OF_FIRST_CHOICES = 2
CHOICES_AVG = 2
DEPTH = 3
COUNT_OF_SCENES = COUNT_OF_FIRST_CHOICES *(CHOICES_AVG * DEPTH) + 2

prompt_structure = f"""
Сгенерируй структуру квеста по следующим правилам:
1. Формат строго:
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум 8 уникальных сцен (не считая финалов/провалов)
   - В стартовой сцене {COUNT_OF_FIRST_CHOICES} выбора
   - В нефинальных сценах может быть 1 или {CHOICES_AVG} выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4
"""
print(prompt_structure)
response_structure = ollama.chat(
    model=MODEL_TO_USE,
    messages=messages_strict + [{"role": "user", "content": prompt_structure}],
    options={"temperature":0.3}
)
structure = response_structure["message"]["content"]
print(structure)


Сгенерируй структуру квеста по следующим правилам:
1. Формат строго: 
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум 8 уникальных сцен (не считая финалов/провалов)
   - В стартовой сцене 2 выбора
   - В нефинальных сценах может быть 1 или 2 выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4

1. start_scene: 1 -> scene1, 2 -> scene2
2. scene1: 1 -> scene2, 2 -> fail1
3. scene2: 1 -> scene3, 2 -> final1
4. scene3: 1 -> fail2
5. scene4: 1 -> scene6, 2 -> fail3
6. scene5: 1 -> scene7, 2 -> fail4
7. scene6: 1 -> fail5, 2 -> final2
8. scene7: 1 -> final3

Повторные сцены и финалы могут быть использованы для разных веток выбор

In [49]:
extracted_info = {
    "setting": "Киберпанк. Все люди давно носят киберимпланты, но до сих пор не могут научиться не кидать окурки на улице... ",
    "main_charachter": "Хакер-одиночка, который не может найти себе место в жизни. Любит пиццу.",
    "goal": "Взломать замок на двери, найти и забрать чип с вирусом. Вирус нужен для ограбления самого крупного в мире банка"
}


json_strict = f"""
Преврати структуру квеста в JSON строго по этим правилам:
1. Сеттинг: {extracted_info["setting"]}
2. Главный герой: {extracted_info["main_charachter"]}
3. Финальная цель: {extracted_info["goal"]}
Правила генерации:
- Сохрани ВСЕ оригинальные scene_id из структуры и связи между сценами
- Для каждой сцены добавь:
  - "text": описание ситуации (2-3 предложения)
  - "choices": массив выборов (количество соответствует заданной ранее структуре)
- Требования к выборам:
  -выборы должны учитывать описание ГГ, окружающую обстановку и элемент случайности
  -Каждый выбор - конкретное действие (глагол + дополнение)
-"after": немедленное последствие выбора (1 предложение)
- Провалы (failX) должны быть в рамках сеттинга и ГГ
- Финалы (finalX) должны соотноситься с целью: """ + extracted_info["goal"] + """
Формат строго JSON, где сцены выводятся через запятую, без комментариев. Пример формата одной сцены:
{
 "start_scene"{
  "scene_id": "start_scene",
  "text": "...",
  "choices": [
      {
      "text": "...",
      "after": "...",
      "next_scene_id": "scene1"
      },
      {
      "text": "...",
      "after": "...",
      "next_scene_id": "scene2"
      }
    ]
  }
}
"""
messages_strict_json = [{
        "role": "system",
        "content": json_strict}]
response_json = ollama.chat(
    model=MODEL_TO_USE,
    options = {"temperature": 0.7},
    messages=messages_strict_json + [{"role": "user", "content": structure}],
    format = "json"
)

print(response_json["message"]["content"])

{
    "start_scene": {
        "scene_id": "start_scene",
        "text": "Вам звонит незнакомец, предлагающий работу хакера за крупную сумму. Он сообщает: 'Для начала получите чип с данными в моей квартире на 6 этаже.'\n\nКвартира находится в старом доме, где давно не было ремонта.",
        "choices": [
            {
                "text": "Отправиться немедленно",
                "after": "Вы быстро собираетесь и выдвигаетесь к указанному месту. Сумка с необходимыми инструментами лежит на диване у двери.",
                "next_scene_id": "scene1"
            },
            {
                "text": "Отказаться, сославшись на плохое самочувствие",
                "after": "Человек звонил еще несколько раз, но оставить вас в покое не удается. Он предлагает небольшую денежную компенсацию за ваше время и усилия.",
                "next_scene_id": "scene2"
            }
        ]
    },
    
    "scene1": {
        "scene_id": "scene1",
        "text": "Вы пришли на нужный этаж. Дом пл

In [50]:
import json
import re

quest_json = response_json["message"]["content"]
# Извлечение JSON (на случай, если модель "добавила мусор")
quest_json = json.loads(
    re.search(r'\{.*\}', quest_json, flags=re.DOTALL).group(0)
)

print(quest_json)  # Готовый JSON для работы

{'start_scene': {'scene_id': 'start_scene', 'text': "Вам звонит незнакомец, предлагающий работу хакера за крупную сумму. Он сообщает: 'Для начала получите чип с данными в моей квартире на 6 этаже.'\n\nКвартира находится в старом доме, где давно не было ремонта.", 'choices': [{'text': 'Отправиться немедленно', 'after': 'Вы быстро собираетесь и выдвигаетесь к указанному месту. Сумка с необходимыми инструментами лежит на диване у двери.', 'next_scene_id': 'scene1'}, {'text': 'Отказаться, сославшись на плохое самочувствие', 'after': 'Человек звонил еще несколько раз, но оставить вас в покое не удается. Он предлагает небольшую денежную компенсацию за ваше время и усилия.', 'next_scene_id': 'scene2'}]}, 'scene1': {'scene_id': 'scene1', 'text': 'Вы пришли на нужный этаж. Дом плохо освещен, ваши шаги эхом отражаются в пустых коридорах. Светящиеся цифровые панели кибер-камер отображают вас и соседей.', 'choices': [{'text': 'Прислушаться к шуму, попытать ключ под дверью', 'after': 'Вы слышите зв

In [51]:
# Запись строки в файл
with open("quest.json", "w", encoding="utf-8") as file:
    json.dump(quest_json, file, ensure_ascii=False, indent=4)